In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
import seaborn as sns
import os
from matplotlib.ticker import FuncFormatter
import datetime


## Import Data

In [ ]:
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                   usecols=[1, 2, 3, 4, 5, 7, 8, 9],
                   dtype={
                          'timestamp': 'uint64',
                          'user_id': 'uint32',
                          'content_id': 'uint16',
                          'content_type_id': 'uint8',
                          'task_container_id': 'uint16',
                          'answered_correctly':'uint8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'}
                   )

In [ ]:
train.head()

In [ ]:
## Describe data

#print('The Train Dataset has {} rows and {} columns'.format(train.shape[0], train.shape[1]))
#print('\n')
#print(train.info())

In [ ]:
#train.isnull().sum()
#train.describe()

In [ ]:
## 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

#train.content_type_id.value_counts()

In [ ]:
#print(f'We have {train.content_id.nunique()} content ids in our train set, of which {train[train.content_type_id == 0].content_id.nunique()} are questions.')
#print(f'We have {train.user_id.nunique()} unique users in our train set')

In [ ]:
train = train[train.content_type_id == False].sort_values('timestamp').reset_index(drop = True)

In [ ]:
#train['lag'] = train.groupby('user_id')['answered_correctly'].shift()
#train_test = train[train.user_id == 124]
#train_test
#test_create_feature = train_test.groupby(['user_id','task_container_id'])['answered_correctly'].mean()
#user_feats_df = pd.DataFrame(test_create_feature)
#train_test = pd.concat([train_test, user_feats_df], axis=1)
#user_feats_df['userId_taskId'] = user_feats_df.index
#user_feats_df['user_Id'] = [user_feats_df['userId_taskId'][x][0] for x in range(0,user_feats_df.shape[0])]
#user_feats_df['task_Id'] = [user_feats_df['userId_taskId'][x][1] for x in range(0,user_feats_df.shape[0])]
#user_feats_df.reset_index(drop = True, inplace = True)

In [ ]:
#cum = train.groupby('user_id')['lag'].agg(['cumsum', 'cumcount'])
# User correctness (measure the users' learning progress)
#train['user_correctness'] = cum['cumsum'] / cum['cumcount']
# Drop the 'lag' feature
#train.drop(columns = ['lag'], inplace = True)

In [ ]:
#train[(train.content_type_id == False)].task_container_id.nunique()
#number_answers.to_csv('number_answers.csv', index = False)

In [ ]:
#number_answers['number_answers'] = number_answers.groupby(['user_id','content_id'])['content_id.1'].cumsum()

In [ ]:
#train[(train['user_id'] ==499347415) & (train['content_id'] == 7964)]
#train.loc[train['user_id']==115]

In [ ]:
group1 = train.loc[(train.content_type_id == False), ['task_container_id', 'user_id']].groupby(['task_container_id']).agg(['count'])
group1.columns = ['avg_questions']
group2 = train.loc[(train.content_type_id == False), ['task_container_id', 'user_id']].groupby(['task_container_id']).agg(['nunique'])
group2.columns = ['avg_questions']
group3 = group1 / group2
group3['avg_questions_seen'] = group3.avg_questions.cumsum()

In [ ]:
#train['task_correctly'] = train.groupby(['user_id', 'task_container_id']).answered_correctly.transform('mean').astype(np.float32)
#train['question_answers'] = train.groupby(['user_id']).content_id.transform('nunique').astype(np.uint16)
#train['number_answers'] = train.groupby(['user_id']).content_id.transform('count').astype(np.uint16)
#train['answers_ratio'] = (train['question_answers'] / train['number_answers']).astype(np.float16)
questions_info = train.groupby(['user_id']).content_id.agg(['nunique','count']).astype(np.uint16)
questions_info.columns = ['question_answers_train','number_answers_train']
questions_info['avg_answers_train'] = (questions_info['question_answers_train'] / questions_info['number_answers_train']).astype(np.float16)

In [ ]:
questions_info

In [ ]:
results_u_final = train.loc[train.content_type_id == False, ['user_id','answered_correctly']].groupby(['user_id']).agg(['mean'])
results_u_final.columns = ['answered_correctly_user']

#results_u2_final = train.loc[train.content_type_id == False, ['user_id','prior_question_had_explanation']].groupby(['user_id']).agg(['mean'])
#results_u2_final.columns = ['explanation_mean_user']

task_final = train.loc[train.content_type_id == False, ['task_container_id','answered_correctly']].groupby(['task_container_id']).agg(['mean'])
task_final.columns = ['task_correctly']

In [ ]:
elapsed_mean = train.prior_question_elapsed_time.mean()
train['prior_question_elapsed_time'] = train['prior_question_elapsed_time'].fillna(elapsed_mean)

In [ ]:
time_question = train.groupby(['user_id'])['prior_question_elapsed_time'].agg(['mean'])
time_question.columns = ['time_questions']

### The values to filter the null values.

In [ ]:
#saving value to fillna
elapsed_mean = train.prior_question_elapsed_time.mean()
#prior_mean_user = results_u2_final.explanation_mean_user.mean()

In [ ]:
#task_final_u = train.groupby(['user_id','task_container_id'])['answered_correctly'].mean()
#task_final_u.columns = ['task_correctly_user']

In [ ]:
questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
#questions = questions_df.copy()
#questions.rename(columns = {'question_id' : 'content_id'}, inplace = True)
train = pd.merge(train, questions_df[['question_id', 'part']], left_on='content_id', right_on='question_id', how='left')
del(questions_df)
train[['part','question_id']].astype('uint16')


In [ ]:
#part_count = train.groupby(['content_id'])['answered_correctly'].count()
#part_answers = train.groupby(['content_id'])['answered_correctly'].sum()
#question_correctly = pd.merge(part_count,part_answers, on = 'content_id',  how="left" )
#question_correctly.columns = ['content_answers_count', 'content_answered_correctly']
#question_correctly['content_percenty'] = question_correctly.content_answered_correctly / question_correctly.content_answers_count
#question_correctly.to_csv('question_correctly.csv', index = True)

In [ ]:
train["part_detail"] = train.part.apply(lambda x : 
                                                             "Listening_Image" if x == 1 else
                                                             "Listening_question" if x ==2 else
                                                             "Listening_conversation" if x ==3 else
                                                             "Listening_conversation" if x == 4 else
                                                             "Reading_grammar" if x == 5 else
                                                             "Reading_fill_in " if x == 6 else
                                                             "Reading_mail"
                                                            )

In [ ]:
question_correctly = pd.read_csv('../input/feature/question_correctly.csv')
#part_user = pd.read_csv('../input/feature/part_user.csv')

# Split the Validation and Train Data

In [ ]:
validation = train.groupby('user_id').tail(10)
train = train[~train.index.isin(validation.index)]
len(train) + len(validation)
#import gc

#cv2_train = pd.read_pickle("../input/cv2-data/cv2_train.pickle")
#cv2_valid = pd.read_pickle("../input/cv2-data/cv2_valid.pickle")




#validation = train[train.row_id.isin(cv2_valid.row_id)]
#train = train[train.row_id.isin(cv2_train.row_id)]

#validation = validation.drop(columns = "row_id")
#train = train.drop(columns = "row_id")

#del cv2_train, cv2_valid
#gc.collect()

In [ ]:
results_u_val = validation.loc[validation.content_type_id == False, ['user_id','answered_correctly']].groupby(['user_id']).agg(['mean'])
results_u_val.columns = ['answered_correctly_user']

#results_u2_val = validation.loc[validation.content_type_id == False, ['user_id','prior_question_had_explanation']].groupby(['user_id']).agg(['mean'])
#results_u2_val.columns = ['explanation_mean_user']

task_val = validation.loc[validation.content_type_id == False, ['task_container_id','answered_correctly']].groupby(['task_container_id']).agg(['mean'])
task_val.columns = ['task_correctly']

time_question_val = validation.groupby(['user_id'])['prior_question_elapsed_time'].agg(['mean'])
time_question_val.columns = ['time_questions']

validation['question_answers'] = validation.groupby(['user_id']).content_id.transform('nunique').astype(np.uint16)
validation['number_answers'] = validation.groupby(['user_id']).content_id.transform('count').astype(np.uint16)
validation['answers_ratio'] = (validation['number_answers'] / validation['question_answers']).astype(np.float16)

In [ ]:
X = train.groupby('user_id').tail(20)
train = train[~train.index.isin(X.index)]
len(X) + len(train) + len(validation)

In [ ]:
results_u_X = X.loc[X.content_type_id == False, ['user_id','answered_correctly']].groupby(['user_id']).agg(['mean'])
results_u_X.columns = ['answered_correctly_user']

#results_u2_X = X.loc[X.content_type_id == False, ['user_id','prior_question_had_explanation']].groupby(['user_id']).agg(['mean'])
#results_u2_X.columns = ['explanation_mean_user']

task_X = X.loc[X.content_type_id == False, ['task_container_id','answered_correctly']].groupby(['task_container_id']).agg(['mean'])
task_X.columns = ['task_correctly']

time_question_X = X.groupby(['user_id'])['prior_question_elapsed_time'].agg(['mean'])
time_question_X.columns = ['time_questions']

X['question_answers'] = X.groupby(['user_id']).content_id.transform('nunique').astype(np.uint16)
X['number_answers'] = X.groupby(['user_id']).content_id.transform('count').astype(np.uint16)
X['answers_ratio'] = (X['number_answers'] / X['question_answers']).astype(np.float16)

In [ ]:
validation.drop(['timestamp', 'content_type_id'], axis=1, inplace=True)
X.drop(['timestamp', 'content_type_id'], axis=1, inplace=True)

# Merging Data

In [ ]:
#task_X
del(train)

In [ ]:
X = pd.merge(X, group3, left_on=['task_container_id'], right_index= True, how="left")
X = pd.merge(X, task_X, left_on=['task_container_id'], right_index= True, how="left")
X = pd.merge(X, results_u_X, on=['user_id'], how="left")
#X = pd.merge(X, results_u2_X, on=['user_id'], how="left")
X = pd.merge(X, question_correctly, on = ['content_id'], how = 'left')
#X = pd.merge(X, part_user, on = ['user_id'], how = 'left' )
X = pd.merge(X, time_question_X, on = ['user_id'], how = 'left' )

In [ ]:
validation.head()

In [ ]:
validation = pd.merge(validation, group3, left_on=['task_container_id'], right_index= True, how="left")
validation = pd.merge(validation, task_val, left_on=['task_container_id'], right_index= True, how="left")
validation = pd.merge(validation, results_u_val, on=['user_id'], how="left")
#validation = pd.merge(validation, results_u2_val, on=['user_id'], how="left")
validation = pd.merge(validation, question_correctly, on = ['content_id'], how = 'left')
#validation = pd.merge(validation, part_user, on = ['user_id'], how = 'left' )
validation = pd.merge(validation, time_question_val, on = ['user_id'], how = 'left' )

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

#X["prior_question_had_explanation"] = X["prior_question_had_explanation"].fillna(False)
#validation["prior_question_had_explanation"] = validation["prior_question_had_explanation"].fillna(False)

#validation["prior_question_had_explanation_enc"] = lb_make.fit_transform(validation["prior_question_had_explanation"])
#X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])

In [ ]:
validation = pd.get_dummies(validation)
X = pd.get_dummies(X)

In [ ]:
y = X['answered_correctly']
X = X.drop(['answered_correctly'], axis=1)

y_val = validation['answered_correctly']
X_val = validation.drop(['answered_correctly'], axis=1)

In [ ]:
X['hmean_user_content_accuracy'] = 2 * (
        (X['answered_correctly_user'] * X['content_percenty']) /
        (X['answered_correctly_user'] + X['content_percenty'])
    )


X_val['hmean_user_content_accuracy'] = 2 * (
        (X_val['answered_correctly_user'] * X_val['content_percenty']) /
        (X_val['answered_correctly_user'] + X_val['content_percenty'])
    )

In [ ]:
#X.isnull().sum()
X_val.info()

In [ ]:
columns = ['answered_correctly_user' ,'avg_questions_seen','content_percenty','hmean_user_content_accuracy',
       'content_answers_count','prior_question_elapsed_time','part_detail_Listening_Image','part_detail_Listening_conversation',
       'part_detail_Listening_question','part_detail_Reading_fill_in','part_detail_Reading_grammar','part_detail_Reading_mail',
           'part','question_answers','number_answers'
      ]
X = X.reindex(columns = columns)
X_val = X_val.reindex(columns = columns)

In [ ]:
#X['answered_correctly_user'].fillna(0.65,  inplace=True)
#X['explanation_mean_user'] = X['explanation_mean_user'].fillna(prior_mean_user)

#X['avg_questions_seen'].fillna(1, inplace = True)
#X['prior_question_elapsed_time'] = X['prior_question_elapsed_time'].fillna(elapsed_mean)
#X['prior_question_had_explanation_enc'].fillna(0, inplace = True)

In [ ]:
#X_val['answered_correctly_user'].fillna(0.65,  inplace=True)
#X_val['explanation_mean_user']= X_val['explanation_mean_user'].fillna(prior_mean_user)

#X_val['avg_questions_seen'].fillna(1, inplace = True)
#X_val['prior_question_elapsed_time'] = X_val['prior_question_elapsed_time'].fillna(elapsed_mean)
#X_val['prior_question_had_explanation_enc'].fillna(0, inplace = True)

In [ ]:
X['part_detail_Reading_fill_in'] = X['part_detail_Reading_fill_in'].fillna(0)
X_val['part_detail_Reading_fill_in'] = X_val['part_detail_Reading_fill_in'].fillna(0)

In [ ]:
X_val.isnull().sum()

# Model

In [ ]:
import lightgbm as lgb

params = {
    'objective': 'binary',
    'metric': 'auc',
    'seed': 42,
    'boosting' : 'gbdt',
    'max_bin': 1500,
    'learning_rate': 0.02,
    'num_leaves': 80
}

lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [ ]:
model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=50,
    num_boost_round=3000,
    early_stopping_rounds=50
)

In [ ]:
#from sklearn.metrics import roc_auc_score

#y_pred = model.predict(X_val)
#y_true = np.array(y_val)
#roc_auc_score(y_true, y_pred)

In [ ]:
lgb.plot_importance(model)
plt.show()

In [ ]:
lgb.plot_importance(model, importance_type = 'gain')
plt.show()

In [ ]:
questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')

In [ ]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop = True)
    #elapsed_mean = test_df.prior_question_elapsed_time.mean()
    test_df['prior_question_elapsed_time'] = test_df['prior_question_elapsed_time'].fillna(elapsed_mean)
    
    test_df = pd.merge(test_df, group3, left_on=['task_container_id'], right_index= True, how="left")
    test_df = pd.merge(test_df, results_u_final, on=['user_id'],  how="left")
    #test_df = pd.merge(test_df, task_final, left_on=['task_container_id'], right_index= True, how="left")
    test_df = pd.merge(test_df, question_correctly, on = ['content_id'], how = 'left')
    test_df["prior_question_had_explanation"] = test_df["prior_question_had_explanation"].fillna(False)
    
    test_df = pd.merge(test_df, questions_df[['question_id', 'part']], left_on='content_id', right_on='question_id', how='left')
    test_df['question_id'].astype('uint16')
    test_df['part'].astype('uint8')
    #time_question = test_df.groupby(['user_id'])['prior_question_elapsed_time'].agg(['mean'])
    #time_question.columns = ['time_questions']
    #test_df = pd.merge(test_df, time_question, on = ['user_id'], how = 'left' )
    #test_df["prior_question_had_explanation_enc"] = lb_make.fit_transform(test_df["prior_question_had_explanation"])
    test_df['number_answers_test'] = test_df.groupby(['user_id']).content_id.transform('count').astype(np.uint16)
    test_df['question_answers_test'] = test_df.groupby(['user_id']).content_id.transform('nunique').astype(np.uint16)
    test_df = pd.merge(test_df, questions_info, on = ['user_id'], how = 'left' )
    test_df['question_answers_train'] = test_df['question_answers_train'].fillna(0)
    test_df['number_answers_train'] = test_df['number_answers_train'].fillna(0)
    test_df['number_answers'] = test_df['number_answers_train']+ test_df['number_answers_test']
    test_df['question_answers'] = test_df['question_answers_train']+ test_df['question_answers_test']
    #question_answers_train	number_answers_train
    test_df['hmean_user_content_accuracy'] = 2 * (
        (test_df['answered_correctly_user'] * test_df['content_percenty']) /
        (test_df['answered_correctly_user'] + test_df['content_percenty'])
    )
    
    test_df["part_detail"] = test_df.part.apply(lambda x : 
                                                             "Listening_Image" if x == 1 else
                                                             "Listening_question" if x ==2 else
                                                             "Listening_conversation" if x ==3 else
                                                             "Listening_conversation" if x == 4 else
                                                             "Reading_grammar" if x == 5 else
                                                             "Reading_fill_in " if x == 6 else
                                                             "Reading_mail"
                                                            )

    test_df = pd.get_dummies(test_df)
    #test_df['part_detail_Reading_fill_in'] = test_df['part_detail_Reading_fill_in'].fillna(0)
    
    test_df['answered_correctly'] =  model.predict(test_df.reindex(columns=columns))
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

In [ ]:
#print(train.answered_correctly.value_counts())

#print('\n')
#ans = train.answered_correctly.value_counts()

#fig = plt.figure(figsize=(12,6))
#ax = ans.plot.bar()
#plt.title("Numer of the answers correctly")
#plt.xticks(rotation=0)
#ax.get_yaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ','))) #add thousands separator
#plt.show()

In [ ]:
#print(train.content_id.value_counts()[:30])

#print('\n')

#content_id = train.content_id.value_counts()[:30]

#fig = plt.figure(figsize=(12,6))
#ax = content_id.plot.bar()
#plt.title("Numer of the content ID")
#plt.xticks(rotation=90)
#ax.get_yaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ','))) #add thousands separator
#plt.show()

In [ ]:
#time_col = ['timestamp', 'prior_question_elapsed_time',]
#plt.figure(figsize=(14,6))
#for i, col in enumerate(time_col):
#    plt.subplot(1, 2, i + 1)
#    train[col].hist(bins = 50,color='red')
#    plt.title(col)
#    plt.tight_layout()